In [ ]:
!pip install py7zr

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 14.1 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 2.4 MB 52.9 MB/s 
     |████████████████████████████████| 43 kB 2.1 MB/s 
     |████████████████████████████████| 131 kB 51.9 MB/s 
     |████████████████████████████████| 59 kB 7.2 MB/s 
     |████████████████████████████████| 357 kB 61.3 MB/s 
     |████████████████████████████████| 2.3 MB 51.9 MB/s 
  Created wheel for py7zr: filename=py7zr-0.18.9-py3-none-any.whl size=67806 sha256=b4b0cf933d5a18018bffc2fd3ae8d6b50f115646f84128e588fad130029de22d
  Stored in directory: /root/.cache/pip/wheels/a7/ce/25/7181dbeffbdb3d6b97a37f02deafdddcaa7ec6d5a5c7b0cabc
Successfully built py7zr


In [ ]:
import py7zr
import os
import PIL
import PIL.Image
from PIL import Image
from glob import glob
import cv2
import numpy as np
import tensorflow as tf
from tqdm import tqdm

In [ ]:
#archive = py7zr.SevenZipFile('/content/annotations_norebars_Pdataset.7z', mode='r')
#archive.extractall()

archive1 = py7zr.SevenZipFile('/content/rebars_annotations.7z', mode='r')
archive1.extractall()

#Checking what are the unique pixel values from this batch of annotations

In [ ]:
masks = sorted(glob(os.path.join("/content/annotations/*")))
unique_pix = []

for i in masks:
  H = 256
  W = 256
  #msk = cv2.imread(i, 0)
  msk = Image.open(i)
  #msk = cv2.resize(msk, (W, H))
  #train_masks.append(msk)
  print('Unique class values for : ', i, 'is', np.unique(msk))
  unique_pix.append(np.unique(msk))

#print(np.unique(unique_pix))

Unique class values for :  /content/annotations/S_bricks_0.png is [0 1]
Unique class values for :  /content/annotations/S_bricks_1.png is [0 1]
Unique class values for :  /content/annotations/S_bricks_10.png is [0 1]
Unique class values for :  /content/annotations/S_bricks_11.png is [0 1]
Unique class values for :  /content/annotations/S_bricks_12.png is [0 1]
Unique class values for :  /content/annotations/S_bricks_13.png is [0 1]
Unique class values for :  /content/annotations/S_bricks_14.png is [0 1]
Unique class values for :  /content/annotations/S_bricks_2.png is [0 1]
Unique class values for :  /content/annotations/S_bricks_3.png is [0 1]
Unique class values for :  /content/annotations/S_bricks_4.png is [0 1]
Unique class values for :  /content/annotations/S_bricks_5.png is [0 1]
Unique class values for :  /content/annotations/S_bricks_6.png is [0 1]
Unique class values for :  /content/annotations/S_bricks_7.png is [0 1]
Unique class values for :  /content/annotations/S_bricks_8.

#Function for transforming pixels in the masks

In [ ]:
def mask_transform(path, classpix):
    mask = Image.open(path)
    name = path.split("/")[-1].split(".")[0]
    w, h = mask.size
    newmask = Image.new('L', (w, h))
    #newmask = Image.new()
    class_pix = 0 
    background_pix = 0
    for i in range(w):
      for j in range(h):
        current_color = mask.getpixel( (i,j) )
        #new_color = current_color
        ####################################################################
        # Creating a new (R,G,B) tuple called new_color
        if current_color == 1: #label created in labelme
          new_color = classpix #pvc pipes = 1; cement = 2; bricks = 3; plastic wire = 4; rebars = 5
          class_pix = class_pix + 1 
        elif current_color == 0:
          new_color = 0
          background_pix = background_pix + 1
        #elif current_color == 2:
        #  new_color = 2
        newmask.putpixel((i, j), new_color)
    if os.path.exists("/content/fixed_annotations"):
        pass
    else:
        os.makedirs("/content/fixed_annotations")
    save_path = f"/content/fixed_annotations/{name}.png"
    newmask.save(save_path)
    print("Number of pixels for class : ", class_pix)
    print("Number of pixels for background : ", background_pix)
    return newmask

#Executing mask_transform

In [ ]:
if __name__ == "__main__":
    """ Seeding """
    #np.random.seed(42)
    #tf.random.set_seed(42)

    """ Load Images """
    test_images = glob("/content/annotations/*.png")
    print(f"Annotation Images: {len(test_images)}")

    classpix = 3 #pvc pipes = 1; cement = 2; bricks = 3; plastic wire = 4; rebars = 5

    #time_taken = []
    for path in tqdm(test_images, total=len(test_images)):
        """ Extract the name """
        #name = path.split("/")[-1].split(".")[0]
        """ Transform existing masks"""
        mask_transform(path, classpix)

modified_masks = glob("fixed_annotations/*.png")
print(f"Modified Images: {len(modified_masks)}")

print("Class values in the dataset are ... ", np.unique(modified_masks))


Annotation Images: 15


  7%|▋         | 1/15 [00:01<00:18,  1.33s/it]

Number of pixels for class :  289454
Number of pixels for background :  285716


 13%|█▎        | 2/15 [00:02<00:18,  1.46s/it]

Number of pixels for class :  359383
Number of pixels for background :  330341


...

Number of pixels for class :  247869
Number of pixels for background :  178983


 93%|█████████▎| 14/15 [00:17<00:01,  1.28s/it]

Number of pixels for class :  443400
Number of pixels for background :  247692


100%|██████████| 15/15 [00:19<00:00,  1.28s/it]

Number of pixels for class :  327026
Number of pixels for background :  340615
Modified Images: 15
Class values in the dataset are ...  ['fixed_annotations/S_bricks_0.png' 'fixed_annotations/S_bricks_1.png'
 'fixed_annotations/S_bricks_10.png' 'fixed_annotations/S_bricks_11.png'
 'fixed_annotations/S_bricks_12.png' 'fixed_annotations/S_bricks_13.png'
 'fixed_annotations/S_bricks_14.png' 'fixed_annotations/S_bricks_2.png'
 'fixed_annotations/S_bricks_3.png' 'fixed_annotations/S_bricks_4.png'
 'fixed_annotations/S_bricks_5.png' 'fixed_annotations/S_bricks_6.png'
 'fixed_annotations/S_bricks_7.png' 'fixed_annotations/S_bricks_8.png'
 'fixed_annotations/S_bricks_9.png']


#Check new unique values

In [ ]:
new_masks = sorted(glob(os.path.join("/content/fixed_annotations/*")))
unique_newpix = []

for i in new_masks:
  #H = 256
  #W = 256
  #msk = cv2.imread(i, 0)
  msk = Image.open(i)
  #msk = cv2.resize(msk, (W, H))
  #train_masks.append(msk)
  print('Unique class values : ', np.unique(msk))
  print('Image size is : ', msk.size)
  unique_newpix.append(np.unique(msk))


print('Unique values from entire dataset : ', np.unique(unique_newpix))

Unique class values :  [0 3]
Image size is :  (972, 711)
Unique class values :  [0 3]
Image size is :  (1186, 427)
Unique class values :  [0 3]
Image size is :  (852, 615)
Unique class values :  [0 3]
Image size is :  (1069, 382)
Unique class values :  [0 3]
Image size is :  (1018, 565)
Unique class values :  [0 3]
Image size is :  (969, 689)
Unique class values :  [0 3]
Image size is :  (877, 705)
Unique class values :  [0 3]
Image size is :  (865, 492)
Unique class values :  [0 3]
Image size is :  (852, 501)
Unique class values :  [0 3]
Image size is :  (1068, 368)
Unique class values :  [0 3]
Image size is :  (1017, 506)
Unique class values :  [0 3]
Image size is :  (822, 719)
Unique class values :  [0 3]
Image size is :  (1071, 644)
Unique class values :  [0 3]
Image size is :  (857, 716)
Unique class values :  [0 3]
Image size is :  (1090, 659)
Unique values from entire dataset :  [0 3]


In [ ]:
with py7zr.SevenZipFile('/content/fixed_annotations.7z', 'w') as archive:
    archive.writeall('/content/fixed_annotations/')